In [1]:
import pandas as pd
from Bio import SeqIO 
import os,sys
import sgRNA_utils.sgRNA_primer_util as su
import module.sequencing_primer as sr
import module.from_gbFile_to_seq as fq
import module.parser_input_to_df as pf
import module.product_and_decorate_editingSeq as p_d_seq
import module.order as order
import warnings   
warnings.filterwarnings('ignore')  
import configparser
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import json
from sgRNA_utils.sgRNA_primer_config import config 
from loguru import logger

# input_path = '/home/yanghe/program/edit_sequence_design/input/config.json'

In [43]:
data = {

        "chopchop_input": "/home/yanghe/tmp/data_preprocessing/output/info_input.csv",   
        "sgRNA_result_path": "/home/yanghe/tmp/chopchop/output/sgRNA.csv",
        "edit_sequence_design_workdir":"/home/yanghe/tmp/edit_sequence_design/output/",
        "ref_genome":"/home/yanghe/program/data_preprocessing/input/GCA_000011325.1_ASM1132v1_genomic.fna",

        "one_plasmid_file_path": "/home/yanghe/program/edit_sequence_design/input/pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        "no_ccdb_plasmid":"/home/yanghe/program/edit_sequence_design/input/no-ccdb-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        "no_sgRNA_plasmid":"/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",

        "uha_dha_config": {
            "max_right_arm_seq_length": 1050,
            "max_left_arm_seq_length": 1050,
            "min_left_arm_seq_length": 1000,   
            "min_right_arm_seq_length": 1000
        },

        
        "plasmid_label":{
            "ccdb_label":"ccdB",
            "promoter_terminator_label":"gRNA",
            "n_20_label":"N20"
        },
        
        "sgRNA_primer_json":{
            "primer3":"AACTATTTATCCAGTTGGTACAAAC"
        },
        "ccdb_primer_json":{
            "primer3":"AACTGATTCAGTCTGATTTCGCGGT"
        },
    
        "sgRNA_region_json":{
            "region1":"tgtgtggaattgtgagcggataacaatttcacacaggaaacagaatt"
        },
        
        "ccdb_region_json":{
            "region1":"ATTGTGAGCGGATAACAATTTCACACAGGAAACAGAATTAATTAAGCTTAAAGGAGTTGAGAATGGATAAGAAATACTCAATAGGCTTAGATATCGGCACAAATAGCGTCGGATGGGCGGTGATC"
        },
        
        "enzyme":{
            "enzyme_name":"BsaI",
            "gap_sequence":"A",
            "protection_sequence":"CCA"
        },
        
        "UHA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "SEQ_ALTERED_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,  
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "DHA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "UP_SGRNA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "DOWN_SGRNA_ARGS": {
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },

        "PLASMID_Q_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,  
            "PRIMER_MAX_TM": 75,    
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "GENOME_Q_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,  
            "PRIMER_MAX_TM": 75,    
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        }
       
    }

In [44]:
#uha_dha_primer
def extract_uha_dha_primer(info_input_df, sgRNA):

    primer_template_for_u_d_ha_df = p_d_seq.create_primer_template(info_input_df,sgRNA)
    uha_dha_primer_df = p_d_seq.design_primer(primer_template_for_u_d_ha_df,'Name_Region','primer_template','u_d')
    uha_dha_primer_df = uha_dha_primer_df.rename(columns={'Region':'id'})
    uha_dha_primer_df = uha_dha_primer_df.join(uha_dha_primer_df.id.str.split(';',expand=True).rename(columns = {0:'Name',1:'Region',2:'Type'})).drop(columns='id')
    
    return uha_dha_primer_df

#uha_dha
def extract_uha_dha(info_input_df,uha_dha_primer_df,sgRNA):
    #整合进突变信息
    info_df = info_input_df[['name','region','seq_altered','type','ref','strand']].rename(columns={'name':'Name','region':'Region'})
    info_df.seq_altered.fillna('',inplace=True)
    uha_dha_info_primer_df = pd.merge(info_df,uha_dha_primer_df,on=['Name','Region'])

    #提取源生同源臂
    uha_dha_df = p_d_seq.create_uha_dha_df(uha_dha_primer_df) 
    #合并突变信息
    uha_dha_df = pd.merge(uha_dha_df,info_df,on=['Name','Region'])
    uha_dha_sgRNA_df = pd.merge(uha_dha_df,sgRNA,on=['Name','Region'],how='inner')

    return uha_dha_info_primer_df, uha_dha_df, uha_dha_sgRNA_df,info_df


def read_chopchopInput_add_uha_dha(genome_path,chopchop_input,uha_dha_params):
    max_left_arm_seq_length = uha_dha_params['max_left_arm_seq_length']
    max_right_arm_seq_length = uha_dha_params['max_right_arm_seq_length']

    info_input_df = su.del_Unnamed(pd.read_csv(chopchop_input))

    def work(mun_id,geneid, mutation_pos_index):
        if mutation_pos_index - max_left_arm_seq_length < 0:
            error_message = "The length of upstream sequence of manipulation site of " + mun_id + " must be larger than sum of 'Max Length of UHA' and 'Max Length of UIS'."
            return error_message,error_message,error_message,error_message

        record = su.extract_seq_from_genome(genome_path,geneid)

        seq_uha_max_whole = str(record[
                        mutation_pos_index - max_left_arm_seq_length : mutation_pos_index
                        ])
        seq_dha_max_whole = str(record[
                                mutation_pos_index : mutation_pos_index + max_right_arm_seq_length
                                ])


        uha_upstream = str(  
                        record[
                            mutation_pos_index - max_left_arm_seq_length - 100 : mutation_pos_index - max_left_arm_seq_length
                        ]
                    )
        dha_downstream=str(
                        record[
                            mutation_pos_index + max_right_arm_seq_length  : mutation_pos_index + max_right_arm_seq_length  + 100
                        ]
                    )
        return  uha_upstream, dha_downstream, seq_uha_max_whole, seq_dha_max_whole

    info_df = su.lambda2cols(info_input_df,lambdaf=work,in_coln=['id','geneid','mutation_pos_index'],to_colns=['uha_upstream','dha_downstream','seq_uha_max_whole','seq_dha_max_whole'])

    return info_df



def one_plasmid_system_design_by_user_primer(primer_json,primer_position_json,gb_path,plasmid_backbone,enzyme_df,enzyme_name):

    sgRNA_promoter_terminator_start = 0
    #对引物进行排序,确定所有引物
    primer_dict_df, primers_sum = p_d_seq.sort_compose_primer(sgRNA_promoter_terminator_start,
                                                         primer_json,
                                                         primer_position_json,
                                                         gb_path,
                                                         plasmid_backbone,
                                                         plasmid_type='one')   
    #给引物加接头
    plasmid_primer_joint_df = p_d_seq.add_joint_plasmid_primer(enzyme_df,enzyme_name,
                                                                 primer_dict_df,
                                                                 int(primers_sum/2),
                                                                 primer_type='ccdb')
    #修改df格式
    plasmid_primer_df = p_d_seq.plasmid_primer(plasmid_primer_joint_df)
    #添加索引
    plasmid_primer_df.insert(0,'index',plasmid_primer_df.index)
    plasmid_primer_df.reset_index(drop=True, inplace=True)
        
    #给引物加产物
    plasmid_primer_df = p_d_seq.add_product_and_size(gb_path, plasmid_primer_df, enzyme_df,enzyme_name,seq=plasmid_backbone)

    return  plasmid_primer_df




def one_plasmid_system_design_by_user_region(uha_dha_sgRNA_df,sgRNA_region_seq_json,one_plasmid_file_path,sgRNA_plasmid_backbone,enzyme_df,enzyme_name):
    
    #序列转换成坐标
    region_json = su.convert_seq_cor(one_plasmid_file_path, sgRNA_region_seq_json,seq=sgRNA_plasmid_backbone)
    #坐标转转换成距离
    distance_dict = region_2_distance(len(sgRNA_plasmid_backbone), region_json, 0)

    #根据区域设计引物
    primer_result_list = p_d_seq.design_primer_by_region_in_plasmid(0,sgRNA_plasmid_backbone,distance_dict)
    plasmid_primer_df = su.result_primer_list_to_df(primer_result_list)

    #加接头
    plasmid_primer_joint_df = p_d_seq.sgRNA_sgRNAprimer_merge(uha_dha_sgRNA_df,plasmid_primer_df,sgRNA_columns=['Name','Region'])
    primers_sum = len(plasmid_primer_df)
    plasmid_primer_joint_df = p_d_seq.add_joint_plasmid_primer(enzyme_df, enzyme_name, plasmid_primer_joint_df, primers_sum, primer_type='ccdb')
    plasmid_primer_joint_df = plasmid_primer_joint_df.drop(columns='Region').drop_duplicates()
    #修改df格式
    plasmid_primer_df = p_d_seq.plasmid_primer(plasmid_primer_joint_df)
    #添加索引
    plasmid_primer_df.insert(0,'index',plasmid_primer_df.index)
    plasmid_primer_df.reset_index(drop=True, inplace=True)

    plasmid_primer_df = p_d_seq.add_product_and_size(one_plasmid_file_path, plasmid_primer_df, enzyme_df, enzyme_name=enzyme_name,seq=sgRNA_plasmid_backbone)

    return  plasmid_primer_df


def compute_distance_in_gb(gb_len,first,last):
    if last < first:
        seq_len = gb_len - first + last
    else:
        seq_len = last - first
    return seq_len


def region_2_distance(sgRNA_plasmid_seq_len, sgRNA_region_json, first_primer_start_position):

    distance_dict = {}
    for k,v in sgRNA_region_json.items():
        arr = v.split(',')
        first = int(arr[0])
        last = int(arr[1]) 
        if first  > first_primer_start_position:
            distance = first - first_primer_start_position
        else:
            distance = sgRNA_plasmid_seq_len - first_primer_start_position + first
        distance_dict.update({distance:v})
    sorted_distance = sorted(distance_dict.keys())

    last_distance_dict = {}
    for i in range(len(sorted_distance)):
        v = distance_dict[sorted_distance[i]]
        arr = v.split(',')
        first = int(arr[0])
        last = int(arr[1])
        new_len = compute_distance_in_gb(sgRNA_plasmid_seq_len,first,last)
        
        if i != 0:
            v = distance_dict[sorted_distance[i-1]]
            arr = v.split(',')
            first = int(arr[0])
            last = int(arr[1])
            old_len = compute_distance_in_gb(sgRNA_plasmid_seq_len,first,last)
                
            min_distance = sorted_distance[i] - sorted_distance[i-1] -old_len
        else:
            min_distance = sorted_distance[i]  

        max_distance = min_distance + new_len

        last_distance_dict.update({f'primer{i+1}':(min_distance, max_distance)})

    print(last_distance_dict)

    return last_distance_dict

def one_plasmid_system_pcr_design_primer(gb_path,
                                         info_df,
                                         uha_dha_sgRNA_df,
                                         uha_dha_info_primer_df,
                                         uha_dha_primer_df,
                                         enzyme_df,
                                         enzyme_name,
                                         plasmid_primer_desgin_type,
                                         seq_json):
    
    #创建新的质粒
    uha_dha_sgRNA_df, promoter_terminator_up_promoter_seq, promoter_terminator_down_terminator_seq, type_kind  = p_d_seq.create_new_plasmid(gb_path, uha_dha_sgRNA_df.copy(), ccdb_label='ccdB', promoter_terminator_label='gRNA', n_20_label='N20')

    #设计sgRNA、ccdb质粒引物
    n20up_primer_template = uha_dha_sgRNA_df[['Name','Region','n20_up_template','Target sequence','Rev Target sequence']]
    n20up_primer_template['Region'] = n20up_primer_template['Name'] +';'+ n20up_primer_template['Region']
    n20up_primer_df = p_d_seq.design_primer(n20up_primer_template,'Region','n20_up_template','sgRNA')
    n20up_primer_df = pd.merge(n20up_primer_template[['Region','Target sequence','Rev Target sequence']],n20up_primer_df,on=['Region'],how='inner')

    #ccdb、sgrna质粒引物加接头
    n20up_primer_df = p_d_seq.add_joint_sgRNA_primer(n20up_primer_df,enzyme_df,enzyme_name,'',stype='n20up_primer_joint')


     #质粒引物的设计类型：1---用户指定范围，2----无需用户指定范围，3----用户指定额外引物  
    if plasmid_primer_desgin_type == 2: 
        n20down_primer_template = uha_dha_sgRNA_df[['Name','Region','n20_down_template','Target sequence','Rev Target sequence']]
        n20down_primer_template['Region'] = n20down_primer_template['Name'] +';'+ n20down_primer_template['Region']
        n20down_primer_df = p_d_seq.design_primer(n20down_primer_template,'Region','n20_down_template','sgRNA')
        n20down_primer_df = pd.merge(n20down_primer_template[['Region','Target sequence','Rev Target sequence']],n20down_primer_df,on=['Region'],how='inner')
        #加接头
        n20down_primer_df = p_d_seq.add_joint_sgRNA_primer(n20down_primer_df,enzyme_df,enzyme_name,'',stype='n20down_primer_joint')

    elif plasmid_primer_desgin_type == 1:
        plasmid_backbone = promoter_terminator_down_terminator_seq
        n20down_primer_df =  one_plasmid_system_design_by_user_region(uha_dha_sgRNA_df,seq_json,gb_path,plasmid_backbone,enzyme_df,enzyme_name)
    elif plasmid_primer_desgin_type == 3:
        plasmid_backbone = promoter_terminator_down_terminator_seq
        primer_position_json, sgRNA_failture_primer = p_d_seq.check_locate_primer(plasmid_backbone, seq_json)
        n20down_primer_df = one_plasmid_system_design_by_user_primer(seq_json,primer_position_json,gb_path,plasmid_backbone,enzyme_df,enzyme_name)


    seq_altered_primer_template = info_df[info_df.seq_altered.apply(lambda x:len(x)>120)][['Name','Region','seq_altered']]
    seq_altered_primer_template['Region'] = seq_altered_primer_template['Name'] +';'+ seq_altered_primer_template['Region']
    seq_altered_primer_df = p_d_seq.design_primer(seq_altered_primer_template,'Region','seq_altered',stype='seq_altered')

    #给同源臂引物加接头:uha取promoter_terminator_down_terminator_seq尾部反义4bp，dha取头promoter_terminator_up_promoter_seq正义4bp
    uha_dha_primer_df = p_d_seq.add_joint_sgRNA_primer(uha_dha_info_primer_df,enzyme_df,enzyme_name,promoter_terminator_down_terminator_seq, promoter_terminator_up_promoter_seq, stype = 'u_d_primer_joint')

    #seq_altered_primer加接头
    seq_altered_primer_df = p_d_seq.add_joint_sgRNA_primer(seq_altered_primer_df,enzyme_df,enzyme_name,'',stype='seq_altered_primer_joint')

    #分别提取，修饰后的uha、dha引物
    in_col = ['Name','Region',"primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']
    ou_col = ['Name','Region',"u_primer_f_seq_(5'-3')","u_primer_r_seq_(5'-3')",'UHA','UHA_size',"d_primer_f_seq_(5'-3')","d_primer_r_seq_(5'-3')",'DHA','DHA_size']
    uha_primer_df, dha_primer_df = p_d_seq.create_uha_dha_primer_df(uha_dha_primer_df, in_col, ou_col)

    n20down_primer_p_df = n20down_primer_df[["Region","primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']]
    n20up_primer_p_df = n20up_primer_df[["Region","primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']]
    seq_altered_primer_df = seq_altered_primer_df[["Region","primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']]

    return uha_dha_sgRNA_df,uha_primer_df, dha_primer_df, n20down_primer_p_df, n20up_primer_p_df, seq_altered_primer_df,type_kind






#one plasmid sequencing primer
def one_plasmid_system_sequencing_design_primer(type_kind,uha_dha_sgRNA_df):
    print(type_kind,"1:代表启动子上游的序列大于600bp,同时终止子下游大于600bp","2:代表启动子上游的序列小于600bp","3:代表终止子上游序列小于600bp")

    uha_dha_sgRNA_df



    if type_kind == 3:
        #载体测序
        sequencing_primer_df=uha_dha_sgRNA_df[['Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                    'plasmid', 'promoter_N20_terminator',
                                    'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['UHA']+sequencing_primer_df['seq_altered']+sequencing_primer_df['DHA']+sequencing_primer_df['promoter_N20_terminator_up'] + sequencing_primer_df['promoter_N20_terminator']
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')

    elif type_kind == 1:
        #载体测序
        #uha_dha测序
        sequencing_primer_df=uha_dha_sgRNA_df[['Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                    'plasmid', 'promoter_N20_terminator',
                                    'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['UHA'] + sequencing_primer_df['seq_altered'] + sequencing_primer_df['DHA']
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df1 = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')
        #promoter_N20_terminator测序
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['promoter_N20_terminator']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df2 = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')
        plasmid_sequencing_primer_df = p_d_seq.merge_sequencing_result(plasmid_sequencing_primer_df1, plasmid_sequencing_primer_df2)

    elif type_kind == 2:
        #载体测序
        #uha_dha测序
        sequencing_primer_df=uha_dha_sgRNA_df[[ 'Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                                'plasmid', 'promoter_N20_terminator',
                                                'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['UHA'] + sequencing_primer_df['seq_altered'] + sequencing_primer_df['DHA'] + sequencing_primer_df['promoter_N20_terminator_up'] + sequencing_primer_df['promoter_N20_terminator'] 
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')

    return plasmid_sequencing_primer_df, sequencing_primer_template

# 数据预处理阶段

In [45]:
    chopchop_input = data['chopchop_input']

    #uha_dha参数
    uha_dha_params = data['uha_dha_config']
    
    # enzyme_path = parent_base_path +'/'+ data['enzyme_path']
    

    sgRNA_result_path = data['sgRNA_result_path']
    one_plasmid_file_path = data['one_plasmid_file_path']
    no_ccdb_plasmid = data['no_ccdb_plasmid']
    no_sgRNA_plasmid = data['no_sgRNA_plasmid']
    
    #plasmid label
    ccdb_label = data['plasmid_label']['ccdb_label']
    promoter_terminator_label = data['plasmid_label']['promoter_terminator_label']
    n_20_label = data['plasmid_label']['n_20_label']   
 
    #primer
    sgRNA_primer_json = data['sgRNA_primer_json']
    ccdb_primer_json = data['ccdb_primer_json']



  
    # sgRNA_region_json = data['sgRNA_region_json']
    # ccdb_region_json = data['ccdb_region_json']

    sgRNA_region_seq_json = data['sgRNA_region_json']


    ccdb_region_seq_json = data['sgRNA_region_json']


    #genome
    genome_path = data['ref_genome']

    #配置引物参数
    # config.S_GLOBAL_ARGS = data['S_GLOBAL_ARGS']
    # config.Q_ARGS = data['Q_ARGS']
    config.UHA_ARGS = data['UHA_ARGS']
    config.SEQ_ALTERED_ARGS = data['SEQ_ALTERED_ARGS']
    config.DHA_ARGS = data['DHA_ARGS']
    config.UP_SGRNA_ARGS = data['UP_SGRNA_ARGS']
    config.DOWN_SGRNA_ARGS = data['DOWN_SGRNA_ARGS']

    config.PLASMID_Q_ARGS = data['PLASMID_Q_ARGS']
    config.GENOME_Q_ARGS = data['GENOME_Q_ARGS']   


    #配置输出参数
    output = data['edit_sequence_design_workdir']
    if not os.path.exists(output):
        os.makedirs(output)

In [46]:
# 1.read 编辑序列信息,给chopchop输入加uha、dha信息
info_input_df = read_chopchopInput_add_uha_dha(genome_path, chopchop_input, uha_dha_params)

In [47]:
# 2.
enzyme_path = './input/enzyme.csv'  
enzyme_df = su.del_Unnamed(pd.read_csv(enzyme_path))
enzyme = data['enzyme']
enzyme_name = enzyme['enzyme_name']

In [48]:
# 3.提取用户选择的sgRNA
sgRNA = p_d_seq.extract_sgRNA_from_chopchop(sgRNA_result_path)

In [49]:
# 4.设计源生同源臂引物
uha_dha_primer_df = extract_uha_dha_primer(info_input_df, sgRNA)

{}
{}
{}
{}
{}
{}
{}
{}
{}
{'PRIMER_LEFT_EXPLAIN': 'considered 8, ok 8', 'PRIMER_RIGHT_EXPLAIN': 'considered 408, low tm 132, not in any ok right region 172, ok 104', 'PRIMER_PAIR_EXPLAIN': 'considered 1, ok 1', 'PRIMER_LEFT_NUM_RETURNED': 1, 'PRIMER_RIGHT_NUM_RETURNED': 1, 'PRIMER_INTERNAL_NUM_RETURNED': 0, 'PRIMER_PAIR_NUM_RETURNED': 1, 'PRIMER_PAIR_0_PENALTY': 10.796287109339062, 'PRIMER_LEFT_0_PENALTY': 3.4730231907085454, 'PRIMER_RIGHT_0_PENALTY': 7.323263918630516, 'PRIMER_LEFT_0_SEQUENCE': 'GTTATTTCCGTGCGCGTTGGTGA', 'PRIMER_RIGHT_0_SEQUENCE': 'GCCTGCATTTATAAAGTGCTGACCC', 'PRIMER_LEFT_0': (0, 23), 'PRIMER_RIGHT_0': (1038, 25), 'PRIMER_LEFT_0_TM': 64.52697680929145, 'PRIMER_RIGHT_0_TM': 62.676736081369484, 'PRIMER_LEFT_0_GC_PERCENT': 52.17391304347826, 'PRIMER_RIGHT_0_GC_PERCENT': 48.0, 'PRIMER_LEFT_0_SELF_ANY_TH': 19.610968059260017, 'PRIMER_RIGHT_0_SELF_ANY_TH': 24.27068115227172, 'PRIMER_LEFT_0_SELF_END_TH': 5.11952728907761, 'PRIMER_RIGHT_0_SELF_END_TH': 15.625926837095221, 'P

In [50]:
# 5.提取同源臂
uha_dha_info_primer_df, uha_dha_df, uha_dha_sgRNA_df, info_df = extract_uha_dha(info_input_df,uha_dha_primer_df,sgRNA)

Index(['primer_f_seq_(5'-3')', 'primer_r_seq_(5'-3')', 'primer_f_Tm',
       'primer_r_Tm', 'product_size', 'product_value', 'Name', 'Region',
       'Type'],
      dtype='object')


# 单质粒设计

In [51]:
# 6.执行单质粒系统
gb_path = one_plasmid_file_path
gb = SeqIO.read(gb_path, "genbank")
gb_name = gb.name

In [52]:
    plasmid_primer_desgin_type = 1
    #质粒引物的设计类型：1---用户指定范围，2----无需用户指定范围，3----用户指定额外引物  
    if plasmid_primer_desgin_type == 2:
        #无需用户指定范围
        uha_dha_sgRNA_df,uha_primer_df,dha_primer_df,n20down_primer_p_df,n20up_primer_p_df, seq_altered_p_df,type_kind = one_plasmid_system_pcr_design_primer(                                                                                                                            
                                                                                                                                        gb_path,
                                                                                                                                        info_df,
                                                                                                                                        uha_dha_sgRNA_df,
                                                                                                                                        uha_dha_info_primer_df,
                                                                                                                                        uha_dha_primer_df,
                                                                                                                                        enzyme_df,
                                                                                                                                        enzyme_name)
    elif plasmid_primer_desgin_type == 1:
        #用户指定范围
        uha_dha_sgRNA_df,uha_primer_df,dha_primer_df,n20down_primer_p_df,n20up_primer_p_df, seq_altered_p_df,type_kind = one_plasmid_system_pcr_design_primer(                                                                                                                            
                                                                                                                                        gb_path,
                                                                                                                                        info_df,
                                                                                                                                        uha_dha_sgRNA_df,
                                                                                                                                        uha_dha_info_primer_df,
                                                                                                                                        uha_dha_primer_df,
                                                                                                                                        enzyme_df,
                                                                                                                                        enzyme_name,
                                                                                                                                        plasmid_primer_desgin_type,
                                                                                                                                        sgRNA_region_seq_json
                                                                                                                                        )
    elif plasmid_primer_desgin_type == 3:
         #用户指定额外引物  
        uha_dha_sgRNA_df,uha_primer_df,dha_primer_df,n20down_primer_p_df,n20up_primer_p_df, seq_altered_p_df,type_kind = one_plasmid_system_pcr_design_primer(                                                                                                                            
                                                                                                                                        gb_path,
                                                                                                                                        info_df,
                                                                                                                                        uha_dha_sgRNA_df,
                                                                                                                                        uha_dha_info_primer_df,
                                                                                                                                        uha_dha_primer_df,
                                                                                                                                        enzyme_df,
                                                                                                                                        enzyme_name,
                                                                                                                                        plasmid_primer_desgin_type,
                                                                                                                                        sgRNA_primer_json
                                                                                                                                        )

-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
8949 9413
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
9002 9022
tag: 0 marker 5057 5363
5057 5363 8949 9413
kjfdhsgjkhd
启动子-n20-终止子上游序列： TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTTGCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAGGCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGTAAAAAGGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCAATGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGGACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGAAGATCCTTTGATCTTTTCTACGGGGTCTGA

seq_altered_left_153019_ecoil_ybeL_ins;BA000036.3:153019-153020: CCA GGTCTC A CCAGGTCTCAATGAACAAGGTTGCTCAATATTACC
seq_altered_right_153019_ecoil_ybeL_ins;BA000036.3:153019-153020: GGTCCAGAGT GGTCCAGAGTTTACGGCTCAAACGGGCGTCT ; CCA GGTCTC A
seq_altered_left_Cgl0851_ecoli_pgi_sub;BA000036.3:907755-909378: CCA GGTCTC A CCAGGTCTCAatgaaaaacatcaatccaacgcaga
seq_altered_right_Cgl0851_ecoli_pgi_sub;BA000036.3:907755-909378: GGTCCAGAGT GGTCCAGAGTttaaccgcgccacgctttatagc ; CCA GGTCTC A
Index(['Name', 'Region', 'primer_f_seq_(5'-3')', 'primer_r_seq_(5'-3')',
       'primer_f_Tm', 'primer_r_Tm', 'primer_f_seq_(5'-3')_joint',
       'primer_r_seq_(5'-3')_joint', 'product_value', 'product_size',
       'product_value_joint', 'product_size_joint', 'Type'],
      dtype='object')


In [53]:
#设计质粒测序引物
plasmid_sequencing_primer_df, sequencing_primer_template = one_plasmid_system_sequencing_design_primer(type_kind,uha_dha_sgRNA_df)

1 1:代表启动子上游的序列大于600bp,同时终止子下游大于600bp 2:代表启动子上游的序列小于600bp 3:代表终止子上游序列小于600bp
0 2554
2954
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACGCTG', 'SEQUENCING_PRIMER_1_TM': 61.16590711741475, 'SEQUENCING_PRIMER_2': 'GAGCTGATACCGCTCGCCGC', 'SEQUENCING_PRIMER_2_TM': 65.96384986687252, 'SEQUENCING_PRIMER_3': 'GAGCGCAACTCGGATGATCC', 'SEQUENCING_PRIMER_3_TM': 61.21822976233301, 'SEQUENCING_PRIMER_4': 'CGAACGCCTGCGCAATGGC', 'SEQUENCING_PRIMER_4_TM': 65.74288342382255, 'SEQUENCING_PRIMER_5': 'CGGCTCACATCAAGTCGTGCC', 'SEQUENCING_PRIMER_5_TM': 64.1446146725608}, {})
0 2066
2466
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACGCTG', 'SEQUENCING_PRIMER_1_TM': 61.16590711741475, 'SEQUENCING_PRIMER_2': 'GAGCTGATACCGCTCGCCGC', 'SEQUENCING_PRIMER_2_TM': 65.96384986687252, 'SEQUENCING_PRIMER_3': 'TCGACGTCTGCAGGAAATGGCA', 'SEQUENCING_PRIMER_3_TM': 64.74457393037193, 'SEQUENCING_PRIMER_4': 'TGCGCTACCACAAGATCGTGCT', 'SEQUENCING_PRIMER_4_TM': 64.78597081263268}, {})
0 2059
2459
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACG

In [87]:
#构建生成质粒图谱引物特征的dataframe

,Region,primer_f_seq_(5'-3')_joint,primer_r_seq_(5'-3')_joint,product_value_joint,product_size_joint
0,153019_ecoil_ybeL_insBA000036.3:153019-153020,CCAGGTCTCAATGAACAAGGTTGCTCAATATTACC,GGTCCAGAGTTTACGGCTCAAACGGGCGTCT,CCAGGTCTCAATGAACAAGGTTGCTCAATATTACCGTGAACTGGTT...,503
1,Cgl0851_ecoli_pgi_subBA000036.3:907755-909378,CCAGGTCTCAatgaaaaacatcaatccaacgcaga,GGTCCAGAGTttaaccgcgccacgctttatagc,CCAGGTCTCAatgaaaaacatcaatccaacgcagaccgctgcctgg...,1670


In [93]:
    #标准化，重命名
    df_common_list = su.rename_common_primer_df(n20up_primer_p_df, n20down_primer_p_df, seq_altered_p_df)
    n20up_primer_p_df, n20down_primer_p_df, seq_altered_p_df = df_common_list[0], df_common_list[1], df_common_list[2]
    uha_primer_df, dha_primer_df = su.rename_u_d_primer_df(uha_primer_df, dha_primer_df)

In [ ]:
sequencing_primer_template,uha_primer_df,seq_altered_p_df,dha_primer_df,n20up_primer_p_df,n20down_primer_p_df

In [94]:
def create_plasmid_primer_featrue_df(sequencing_primer_template,
                                     uha_primer_df,
                                     seq_altered_p_df,
                                     dha_primer_df,
                                     n20up_primer_p_df):
    
    sequencing_plasmid = sequencing_primer_template[['Region','plasmid']].rename(columns={'Region':'ID','plasmid':'PLASMID'})
    
    uha_primer_df = uha_primer_df[['ID','PRIMER_LEFT_WHOLE_SEQUENCE','PRIMER_RIGHT_WHOLE_SEQUENCE']]
    uha_primer_df.columns = ['ID','UHA_PRIMER_LEFT_WHOLE_SEQUENCE','UHA_PRIMER_RIGHT_WHOLE_SEQUENCE']
    
    seq_altered_p_df = seq_altered_p_df[['ID','PRIMER_LEFT_WHOLE_SEQUENCE','PRIMER_RIGHT_WHOLE_SEQUENCE']]
    seq_altered_p_df.columns = ['ID','SEQ_PRIMER_LEFT_WHOLE_SEQUENCE','SEQ_PRIMER_RIGHT_WHOLE_SEQUENCE']
    
    dha_primer_df = dha_primer_df[['ID','PRIMER_LEFT_WHOLE_SEQUENCE','PRIMER_RIGHT_WHOLE_SEQUENCE']]
    dha_primer_df.columns = ['ID','DHA_PRIMER_LEFT_WHOLE_SEQUENCE','DHA_PRIMER_RIGHT_WHOLE_SEQUENCE']
    
    n20up_primer_p_df = n20up_primer_p_df[['ID','PRIMER_LEFT_WHOLE_SEQUENCE','PRIMER_RIGHT_WHOLE_SEQUENCE']]
    n20up_primer_p_df.columns = ['ID','N20UP_PRIMER_LEFT_WHOLE_SEQUENCE','N20UP_PRIMER_RIGHT_WHOLE_SEQUENCE']
    
#     n20down_primer_p_df = n20down_primer_p_df[['ID','PRIMER_LEFT_WHOLE_SEQUENCE','PRIMER_RIGHT_WHOLE_SEQUENCE']]
#     n20down_primer_p_df.columns = ['ID','N20DOWN_PRIMER_LEFT_WHOLE_SEQUENCE','N20DOWN_PRIMER_RIGHT_WHOLE_SEQUENCE']
    
    df = pd.merge(sequencing_plasmid,uha_primer_df).merge(dha_primer_df).merge(n20up_primer_p_df)
    df = pd.merge(seq_altered_p_df,df,how='outer')
    
    return df

In [95]:
import numpy as np
plasmid_primer_featrue_df = create_plasmid_primer_featrue_df(sequencing_primer_template,
                                                                 uha_primer_df,
                                                                 seq_altered_p_df,
                                                                 dha_primer_df,
                                                                 n20up_primer_p_df)
plasmid_primer_featrue_df = plasmid_primer_featrue_df.fillna('')


In [96]:
def create_primerCor_in_plasmid(plasmid_seq,primer):
    start = plasmid_seq.find(primer)
    end = start + len(primer)
    return start,end

In [97]:
def columns_to_row(n20down_primer_p_df):
    n20down_primer_p_df = n20down_primer_p_df.sort_values(by='ID')
    temp = n20down_primer_p_df[['PRIMER_LEFT_WHOLE_SEQUENCE','PRIMER_RIGHT_WHOLE_SEQUENCE']]
    index = [ str(i[0])+'_'+str(i[1]) for i in temp.stack().to_frame().index]
    temp = temp.stack().to_frame()
    temp.index = index
    temp = temp.T
    return temp

In [84]:
li=['1','2','3','4']

In [86]:
','.join(li)

'1,2,3,4'

In [114]:
sgRNA_enzyme_df = enzyme_df[enzyme_df['name']==enzyme_name]
protective_base = sgRNA_enzyme_df.loc[0,'protective_base']
recognition_seq = sgRNA_enzyme_df.loc[0,'recognition_seq']
cut_seq_len = sgRNA_enzyme_df.loc[0,'cut_seq_len']
gap_len = sgRNA_enzyme_df.loc[0,'gap_len']
joint_len = len(protective_base) + len(recognition_seq) + gap_len + cut_seq_len

n20down_primer_temp_df = columns_to_row(n20down_primer_p_df)


#做标配
for i,v in plasmid_primer_featrue_df.iterrows():
    plasmid  = v['PLASMID'].upper()
    uha_left_primer = v['UHA_PRIMER_LEFT_WHOLE_SEQUENCE']
    uha_left_nojoint_primer = uha_left_primer[joint_len:].upper()
    uha_left_primer_cor = create_primerCor_in_plasmid(plasmid,uha_left_nojoint_primer)
    uha_left_primer_cor = uha_left_primer_cor[0] - joint_len, uha_left_primer_cor[1]
    
    uha_right_primer = v['UHA_PRIMER_RIGHT_WHOLE_SEQUENCE']
    uha_right_nojoint_primer = uha_right_primer[joint_len:].upper()
    uha_right_primer_cor = create_primerCor_in_plasmid(plasmid, su.revComp(uha_right_nojoint_primer))
    uha_right_primer_cor = uha_right_primer_cor[0], uha_right_primer_cor[1] + joint_len
    
    dha_left_primer = v['DHA_PRIMER_LEFT_WHOLE_SEQUENCE']
    dha_left_nojoint_primer = dha_left_primer[joint_len:].upper()
    dha_left_primer_cor = create_primerCor_in_plasmid(plasmid, dha_left_nojoint_primer)
    dha_left_primer_cor = dha_left_primer_cor[0] - joint_len, dha_left_primer_cor[1]
    
    dha_right_primer = v['DHA_PRIMER_RIGHT_WHOLE_SEQUENCE']
    dha_right_nojoint_primer = dha_right_primer[joint_len:].upper()
    dha_right_primer_cor = create_primerCor_in_plasmid(plasmid, su.revComp(dha_right_nojoint_primer))
    dha_right_primer_cor = dha_right_primer_cor[0] - joint_len, dha_right_primer_cor[1]
    
    seq_altered_left_primer = v['SEQ_PRIMER_LEFT_WHOLE_SEQUENCE']
    seq_altered_left_primer_cor = ()
    if seq_altered_left_primer != '':
        seq_altered_left_nojoint_primer = seq_altered_left_primer[joint_len-cut_seq_len:].upper()
        seq_altered_left_primer_cor = create_primerCor_in_plasmid(plasmid, seq_altered_left_nojoint_primer)
        seq_altered_left_primer_cor = seq_altered_left_primer_cor[0] - (joint_len-cut_seq_len), seq_altered_left_primer_cor[1]
        
    seq_altered_right_primer =  v['SEQ_PRIMER_RIGHT_WHOLE_SEQUENCE']   
    seq_altered_right_primer_cor = ()
    if seq_altered_right_primer != '':
        seq_altered_right_nojoint_primer = seq_altered_right_primer[joint_len-cut_seq_len:].upper()
        seq_altered_right_primer_cor = create_primerCor_in_plasmid(plasmid, su.revComp(seq_altered_right_nojoint_primer))
        seq_altered_right_primer_cor = seq_altered_right_primer_cor[0], seq_altered_right_primer_cor[1] + (joint_len-cut_seq_len)
    
    n20up_left_primer = v['N20UP_PRIMER_LEFT_WHOLE_SEQUENCE']
    n20up_left_nojoint_primer = n20up_left_primer[joint_len-cut_seq_len:].upper()
    n20up_left_primer_cor = create_primerCor_in_plasmid(plasmid, n20up_left_nojoint_primer)
    n20up_left_primer_cor = n20up_left_primer_cor[0]-(joint_len-cut_seq_len), n20up_left_primer_cor[1]
    
    n20up_right_primer = v['N20UP_PRIMER_RIGHT_WHOLE_SEQUENCE']
    n20up_right_nojoint_primer = n20up_right_primer[joint_len-cut_seq_len+20:].upper()
    n20up_right_primer_cor = create_primerCor_in_plasmid(plasmid, su.revComp(n20up_right_nojoint_primer))
    n20up_right_primer_cor = n20up_right_primer_cor[0], n20up_left_primer_cor[1]+(joint_len-cut_seq_len+20)
     
    
    #在质粒上找到位置坐标
    n20down_primer_temp_df = columns_to_row(n20down_primer_p_df)
    n20down_primer_cor_dict={}
    for i in n20down_primer_temp_df.columns:
        value = n20down_primer_temp_df.loc[0,i]
        temp_value = value[joint_len-cut_seq_len:]
        
        if 'RIGHT' in i:
            primer_cor = create_primerCor_in_plasmid(plasmid, su.revComp(temp_value))
            n20down_right_primer_cor = primer_cor[0], primer_cor[1] + (joint_len-cut_seq_len)
            n20down_primer_cor_dict.update({i: f"{n20down_right_primer_cor};{value}"})
        else:
            primer_cor = create_primerCor_in_plasmid(plasmid,temp_value)
            n20down_left_primer_cor = primer_cor[0] - (joint_len-cut_seq_len), primer_cor[1]
            n20down_primer_cor_dict.update({i:f'{n20down_left_primer_cor};{value}'})

            
    #生成gb文件
#     print(uha_left_primer_cor,uha_left_primer,
#           uha_right_primer_cor,uha_right_primer,'\n',
#           dha_left_primer_cor,dha_left_primer,
#           dha_right_primer_cor,dha_right_primer,'\n',
#           seq_altered_left_primer_cor, seq_altered_left_primer,
#           seq_altered_right_primer_cor, seq_altered_right_primer,'\n',
#           n20up_left_primer_cor,n20up_left_primer,
#           n20up_right_primer_cor,n20up_right_primer,'\n'
#          )
    
#     uha_left_primer_cor = [str(i) in i for uha_left_primer_cor.items()]
#     print(uha_left_primer_cor)
    primer_cor_dict = {
        'UHA_PRIMER_LEFT_WHOLE_SEQUENCE':f'{uha_left_primer_cor};{uha_left_primer}',
        'UHA_PRIMER_RIGHT_WHOLE_SEQUENCE':f'{uha_right_primer_cor};{uha_right_primer}',
        'SEQ_PRIMER_LEFT_WHOLE_SEQUENCE':f'{seq_altered_left_primer_cor};{seq_altered_left_primer}',
        'SEQ_PRIMER_RIGHT_WHOLE_SEQUENCE':f'{seq_altered_right_primer_cor};{seq_altered_right_primer}',
        'DHA_PRIMER_LEFT_WHOLE_SEQUENCE':f'{dha_left_primer_cor};{dha_left_primer}',
        'DHA_PRIMER_RIGHT_WHOLE_SEQUENCE':f'{dha_right_primer_cor};{dha_right_primer}',
        'N20UP_PRIMER_LEFT_WHOLE_SEQUENCE':f'{n20up_left_primer_cor};{n20up_left_primer}',
        'N20UP_PRIMER_RIGHT_WHOLE_SEQUENCE':f"{n20up_right_primer_cor};{n20up_right_primer}"
    }
    
    primer_cor_dict.update(n20down_primer_cor_dict)
    print(primer_cor_dict,'\n')
#     gb = create_gb_file(plasmid,features)
    
    

{'UHA_PRIMER_LEFT_WHOLE_SEQUENCE': '(-14, 20);CCAGGTCTCATCCTAAAGCCCTGCCCACCGGATG', 'UHA_PRIMER_RIGHT_WHOLE_SEQUENCE': '(1009, 1048);GGTCCAGAGTTCATGAACGAGTATTTGTATTTCAATGTC', 'SEQ_PRIMER_LEFT_WHOLE_SEQUENCE': '(1024, 1059);CCAGGTCTCAATGAACAAGGTTGCTCAATATTACC', 'SEQ_PRIMER_RIGHT_WHOLE_SEQUENCE': '(1496, 1527);GGTCCAGAGTTTACGGCTCAAACGGGCGTCT', 'DHA_PRIMER_LEFT_WHOLE_SEQUENCE': '(1503, 1542);CCAGGTCTCAGTAAGCTTTTAAGTTTTCTCGGTGATCCG', 'DHA_PRIMER_RIGHT_WHOLE_SEQUENCE': '(2520, 2554);GGTCCAGAGTCTCACGCGCACGCCTCATGGATTC', 'N20UP_PRIMER_LEFT_WHOLE_SEQUENCE': '(2544, 2575);CCAGGTCTCATGAGAGTTTTCGCCCCGAAGA', 'N20UP_PRIMER_RIGHT_WHOLE_SEQUENCE': '(6169, 2605);GGTCCAGAGTAGAAAACTTAAAAGCGAACGTGAATTACACTGTACCTGTTGCGT', '0_PRIMER_LEFT_WHOLE_SEQUENCE': '(6203, 6237);CCAGGTCTCAGTTTTAGAGCTAGAAATAGCAAGT', '0_PRIMER_RIGHT_WHOLE_SEQUENCE': '(8796, 8830);GGTCCAGAGTGTTATCCGCTCACAATTCCACACA', '1_PRIMER_LEFT_WHOLE_SEQUENCE': '(8806, 8840);CCAGGTCTCATAACAATTTCACACAGGAAACAGA', '1_PRIMER_RIGHT_WHOLE_SEQUENCE': '(1387

In [ ]:
def create_gb_file(plasmid,primer_cor_dict):
    
    for k,v in primer_cor_dict.items():
        print(k,v)
        cor = v.split(';')[0]
        seq = v.split(';')[1]
        if cor != () or seq != '':
            SeqFeature(FeatureLocation(cor[], 30), type="primer_bind", strand=1, qualifiers={"note": "Left primer"})
    

In [116]:
   
    for k,v in primer_cor_dict.items():
        print(k,v)
        cor = v.split(';')[0]
        seq = v.split(';')[1]
        if cor != '()' or seq != '':
            cor = cor.replace('(','').replace(')',"")
            arr = cor.split(',')
            print(arr[0],arr[1])
            SeqFeature(FeatureLocation( ,), type="primer_bind", strand=1, qualifiers={"note": "Left primer"})
           

UHA_PRIMER_LEFT_WHOLE_SEQUENCE (-14, 20);CCAGGTCTCATCCTTTACCGGACACGCGCAACAC
-14  20
UHA_PRIMER_RIGHT_WHOLE_SEQUENCE (995, 1033);GGTCCAGAGTTGCCTTTTCATGCAGCAAAGAATGATCG
995  1033
SEQ_PRIMER_LEFT_WHOLE_SEQUENCE ();
SEQ_PRIMER_RIGHT_WHOLE_SEQUENCE ();
DHA_PRIMER_LEFT_WHOLE_SEQUENCE (1009, 1047);CCAGGTCTCAGGCACCTGAAATTTCCGTCATGCCGACC
1009  1047
DHA_PRIMER_RIGHT_WHOLE_SEQUENCE (2012, 2046);GGTCCAGAGTCTCACTGCGTGCAGGTCCCTGGTT
2012  2046
N20UP_PRIMER_LEFT_WHOLE_SEQUENCE (2036, 2067);CCAGGTCTCATGAGAGTTTTCGCCCCGAAGA
2036  2067
N20UP_PRIMER_RIGHT_WHOLE_SEQUENCE (5661, 2097);GGTCCAGAGTGAAATTTCCGTCATGCCGACTGAATTACACTGTACCTGTTGCGT
5661  2097
0_PRIMER_LEFT_WHOLE_SEQUENCE (5695, 5729);CCAGGTCTCAGTTTTAGAGCTAGAAATAGCAAGT
5695  5729
0_PRIMER_RIGHT_WHOLE_SEQUENCE (8288, 8322);GGTCCAGAGTGTTATCCGCTCACAATTCCACACA
8288  8322
1_PRIMER_LEFT_WHOLE_SEQUENCE (8298, 8332);CCAGGTCTCATAACAATTTCACACAGGAAACAGA
8298  8332
1_PRIMER_RIGHT_WHOLE_SEQUENCE (13367, 13402);GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA
13367  13402


In [ ]:
sequencing_primer_template[['Region','plasmid']]
uha_primer_df[['Region',"primer_f_seq_(5'-3')","primer_r_seq_(5'-3')"]]
dha_primer_df[['Region',"primer_f_seq_(5'-3')","primer_r_seq_(5'-3')"]]

In [95]:
#标准化，重命名
df_sequencing_list = su.rename_sequencing_primer_df(plasmid_sequencing_primer_df, genome_sequencing_primer_df)
plasmid_sequencing_primer_df, genome_sequencing_primer_df = df_sequencing_list[0], df_sequencing_list[1]

NameError: name 'genome_sequencing_primer_df' is not defined

In [34]:
uha_dha_sgRNA_df.columns

Index(['Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size', 'seq_altered',
       'type', 'ref', 'strand', 'Rank', 'Target sequence', 'Genomic location',
       'Strand', 'GC content (%)', 'Self-complementarity', 'MM0', 'MM1', 'MM2',
       'MM3', 'Efficiency'],
      dtype='object')

### 一、单质粒pcr引物：
#### 输入：gb_path、info_df uha_dha_sgRNA_df、 uha_dha_info_primer_df、 uha_dha_primer_df enzyme_df 、enzyme_name
#### 输出：uha_dha_sgRNA_df,uha_primer_df,dha_primer_df,n20down_primer_p_df,n20up_primer_p_df, seq_altered_primer_df,type_kind

In [8]:
#创建新的质粒
uha_dha_sgRNA_df, promoter_terminator_up_promoter_seq, promoter_terminator_down_terminator_seq, type_kind  = p_d_seq.create_new_plasmid(gb_path, uha_dha_sgRNA_df.copy(), ccdb_label=ccdb_label, promoter_terminator_label=promoter_terminator_label, n_20_label=n_20_label)

-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
8949 9413
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
9002 9022
tag: 0 marker 5057 5363
5057 5363 8949 9413
kjfdhsgjkhd
启动子-n20-终止子上游序列： TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTTGCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAGGCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGTAAAAAGGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCAATGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGGACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGAAGATCCTTTGATCTTTTCTACGGGGTCTGA

In [9]:
#设计promoter_terminator_up_promoter_seq引物
n20up_primer_template = uha_dha_sgRNA_df[['Name','Region','n20_up_template','Target sequence','Rev Target sequence']]
n20up_primer_template['Region'] = n20up_primer_template['Name'] +';'+ n20up_primer_template['Region']
n20up_primer_df = p_d_seq.design_primer(n20up_primer_template,'Region','n20_up_template','sgRNA')
n20up_primer_df = pd.merge(n20up_primer_template[['Region','Target sequence','Rev Target sequence']],n20up_primer_df,on=['Region'],how='inner')

#设计promoter_terminator_down_terminator_seq引物
n20down_primer_template = uha_dha_sgRNA_df[['Name','Region','n20_down_template','Target sequence','Rev Target sequence']]
n20down_primer_template['Region'] = n20down_primer_template['Name'] +';'+ n20down_primer_template['Region']
n20down_primer_df = p_d_seq.design_primer(n20down_primer_template,'Region','n20_down_template','sgRNA')
n20down_primer_df = pd.merge(n20down_primer_template[['Region','Target sequence','Rev Target sequence']],n20down_primer_df,on=['Region'],how='inner')

#设计编辑区域引物
seq_altered_primer_template = info_df[info_df.seq_altered.apply(lambda x:len(x)>120)][['Name','Region','seq_altered']]
seq_altered_primer_template['Region'] = seq_altered_primer_template['Name'] + seq_altered_primer_template['Region']
seq_altered_primer_df = p_d_seq.design_primer(seq_altered_primer_template,'Region','seq_altered',stype='seq_altered')

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [10]:
#给同源臂引物加接头:uha取promoter_terminator_down_terminator_seq尾部反义4bp，dha取头promoter_terminator_up_promoter_seq正义4bp
uha_dha_primer_df = p_d_seq.add_joint_sgRNA_primer(uha_dha_info_primer_df,enzyme_df,enzyme_name,promoter_terminator_down_terminator_seq, promoter_terminator_up_promoter_seq, stype = 'u_d_primer_joint')

u_d_primer_joint uha_right_153019_ecoil_ybeL_insBA000036.3:153019-153020: GGTCCAGAGT TCAT GGTCCAGAGTTCATGAACGAGTATTTGTATTTCAATGTC ; CCA GGTCTC A
u_d_primer_joint uha_left_153019_ecoil_ybeL_insBA000036.3:153019-153020: CCA GGTCTC A TCCT CCAGGTCTCATCCTAAAGCCCTGCCCACCGGATG
u_d_primer_joint dha_right_153019_ecoil_ybeL_insBA000036.3:153019-153020: GGTCCAGAGT CTCA GGTCCAGAGTCTCACGCGCACGCCTCATGGATTC ; CCA GGTCTC A
u_d_primer_joint uha_right_Cgl0006_1176_G_A_subBA000036.3:6529-6530: GGTCCAGAGT T GGTCCAGAGTTCGCGGACAGGCCTTCACGAC ; CCA GGTCTC A
u_d_primer_joint uha_left_Cgl0006_1176_G_A_subBA000036.3:6529-6530: CCA GGTCTC A TCCT CCAGGTCTCATCCTAAGCGCCCGGGCATGTACAT
u_d_primer_joint dha_left_Cgl0006_1176_G_A_subBA000036.3:6529-6530: CCA GGTCTC A GCG A TTATTTCCGTGCGCGTTGGTGA
u_d_primer_joint dha_right_Cgl0006_1176_G_A_subBA000036.3:6529-6530: GGTCCAGAGT CTCA GGTCCAGAGTCTCAGAAGCCATAGAAGCCTGCAT ; CCA GGTCTC A
u_d_primer_joint uha_left_Cgl0141_cds_delBA000036.3:153222-155613: CCA GGTCTC A TCCT CCAGGTCTC

In [11]:
#promoter_terminator_up_promoter_seq、promoter_terminator_down_terminator_seq引物加接头
n20up_primer_df = p_d_seq.add_joint_sgRNA_primer(n20up_primer_df,enzyme_df,enzyme_name,'',stype='n20up_primer_joint')
n20down_primer_df = p_d_seq.add_joint_sgRNA_primer(n20down_primer_df,enzyme_df,enzyme_name,'',stype='n20down_primer_joint')

n20up_primer_joint n20up_left_153019_ecoil_ybeL_ins;BA000036.3:153019-153020: CCA GGTCTC A CCAGGTCTCATGAGAGTTTTCGCCCCGAAGA
n20up_primer_joint n20up_right_153019_ecoil_ybeL_ins;BA000036.3:153019-153020: GGTCCAGAGT AGAAAACTTAAAAGCGAACG GGTCCAGAGTAGAAAACTTAAAAGCGAACGTGAATTACACTGTACCTGTTGCGT ; CCA GGTCTC A
n20up_primer_joint n20up_left_Cgl0006_1176_G_A_sub;BA000036.3:6529-6530: CCA GGTCTC A CCAGGTCTCATGAGAGTTTTCGCCCCGAAGA
n20up_primer_joint n20up_right_Cgl0006_1176_G_A_sub;BA000036.3:6529-6530: GGTCCAGAGT CGGTTATTTCCGTGCGCGTT GGTCCAGAGTCGGTTATTTCCGTGCGCGTTTGAATTACACTGTACCTGTTGCGT ; CCA GGTCTC A
n20up_primer_joint n20up_left_Cgl0141_cds_del;BA000036.3:153222-155613: CCA GGTCTC A CCAGGTCTCATGAGAGTTTTCGCCCCGAAGA
n20up_primer_joint n20up_right_Cgl0141_cds_del;BA000036.3:153222-155613: GGTCCAGAGT CGGTGTGTTAATCCGCCAGC GGTCCAGAGTCGGTGTGTTAATCCGCCAGCTGAATTACACTGTACCTGTTGCGT ; CCA GGTCTC A
n20up_primer_joint n20up_left_Cgl0591_-1_Ppgk_promoter_ins;BA000036.3:604469-604470: CCA GGTCTC A CCAGGTCTCATG

In [12]:
#seq_altered_primer加接头
seq_altered_primer_df = p_d_seq.add_joint_sgRNA_primer(seq_altered_primer_df,enzyme_df,enzyme_name,'',stype='seq_altered_primer_joint')

seq_altered_left_153019_ecoil_ybeL_insBA000036.3:153019-153020: CCA GGTCTC A CCAGGTCTCAATGAACAAGGTTGCTCAATATTACC
seq_altered_right_153019_ecoil_ybeL_insBA000036.3:153019-153020: GGTCCAGAGT GGTCCAGAGTTTACGGCTCAAACGGGCGTCT ; CCA GGTCTC A
seq_altered_left_Cgl0851_ecoli_pgi_subBA000036.3:907755-909378: CCA GGTCTC A CCAGGTCTCAatgaaaaacatcaatccaacgcaga
seq_altered_right_Cgl0851_ecoli_pgi_subBA000036.3:907755-909378: GGTCCAGAGT GGTCCAGAGTttaaccgcgccacgctttatagc ; CCA GGTCTC A


In [13]:
#分别提取，修饰后的uha、dha引物
in_col = ['Name','Region',"primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']
ou_col = ['Name','Region',"u_primer_f_seq_(5'-3')","u_primer_r_seq_(5'-3')",'UHA','UHA_size',"d_primer_f_seq_(5'-3')","d_primer_r_seq_(5'-3')",'DHA','DHA_size']
uha_primer_df, dha_primer_df = p_d_seq.create_uha_dha_primer_df(uha_dha_primer_df, in_col, ou_col)
n20down_primer_p_df = n20down_primer_df[["Region","primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']]
n20up_primer_p_df = n20up_primer_df[["Region","primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']]


Index(['Name', 'Region', 'primer_f_seq_(5'-3')', 'primer_r_seq_(5'-3')',
       'primer_f_Tm', 'primer_r_Tm', 'primer_f_seq_(5'-3')_joint',
       'primer_r_seq_(5'-3')_joint', 'product_value', 'product_size',
       'product_value_joint', 'product_size_joint', 'Type'],
      dtype='object')


## 二、质粒测序验证引物
### 1:代表启动子上游的序列大于600bp,同时终止子下游大于600bp,2:代表启动子上游的序列小于600bp,3:代表终止子下游序列小于600bp
#### 输入：type_kind,uha_dha_sgRNA_df
#### 输出：plasmid_sequencing_primer_df

In [14]:
print(type_kind,"1:代表启动子上游的序列大于600bp,同时终止子下游大于600bp","2:代表启动子上游的序列小于600bp","3:代表终止子上游序列小于600bp")

1 1:代表启动子上游的序列大于600bp,同时终止子下游大于600bp 2:代表启动子上游的序列小于600bp 3:代表终止子上游序列小于600bp


In [15]:
if type_kind == 2:
        #载体测序策略：将uha_dha、promoter_N20_terminator_up、promoter_N20_terminator一起测序
        sequencing_primer_df=uha_dha_sgRNA_df[['Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                    'plasmid', 'promoter_N20_terminator',
                                    'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['UHA']+sequencing_primer_df['seq_altered']+sequencing_primer_df['DHA']+sequencing_primer_df['promoter_N20_terminator_up'] + sequencing_primer_df['promoter_N20_terminator']
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')

elif type_kind == 1:
        #载体测序策略：将uha_dha、promoter_N20_terminator分开测序
        #uha_dha测序
        sequencing_primer_df=uha_dha_sgRNA_df[['Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                    'plasmid', 'promoter_N20_terminator',
                                    'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['UHA'] + sequencing_primer_df['seq_altered'] + sequencing_primer_df['DHA']
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df1 = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')
        #promoter_N20_terminator测序
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['promoter_N20_terminator']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df2 = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')
        plasmid_sequencing_primer_df = p_d_seq.merge_sequencing_result(plasmid_sequencing_primer_df1, plasmid_sequencing_primer_df2)

elif type_kind == 3:
        #载体测序策略：将promoter_N20_terminator、promoter_N20_terminator_down、uha_dha一起测序
        sequencing_primer_df=uha_dha_sgRNA_df[[ 'Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                                'plasmid', 'promoter_N20_terminator',
                                                'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['promoter_N20_terminator'] + sequencing_primer_df['promoter_N20_terminator_down'] + sequencing_primer_df['UHA'] + sequencing_primer_df['seq_altered'] + sequencing_primer_df['DHA']
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')


0 2554
2954
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACGCTG', 'SEQUENCING_PRIMER_1_TM': 61.16590711741475, 'SEQUENCING_PRIMER_2': 'GAGCTGATACCGCTCGCCGC', 'SEQUENCING_PRIMER_2_TM': 65.96384986687252, 'SEQUENCING_PRIMER_3': 'GAGCGCAACTCGGATGATCC', 'SEQUENCING_PRIMER_3_TM': 61.21822976233301, 'SEQUENCING_PRIMER_4': 'CGAACGCCTGCGCAATGGC', 'SEQUENCING_PRIMER_4_TM': 65.74288342382255, 'SEQUENCING_PRIMER_5': 'CGGCTCACATCAAGTCGTGCC', 'SEQUENCING_PRIMER_5_TM': 64.1446146725608}, {})
0 2077
2477
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACGCTG', 'SEQUENCING_PRIMER_1_TM': 61.16590711741475, 'SEQUENCING_PRIMER_2': 'GAGCTGATACCGCTCGCCGC', 'SEQUENCING_PRIMER_2_TM': 65.96384986687252, 'SEQUENCING_PRIMER_3': 'TCGACGTCTGCAGGAAATGGCA', 'SEQUENCING_PRIMER_3_TM': 64.74457393037193, 'SEQUENCING_PRIMER_4': 'ATCGTGCTGATGGCCGACGC', 'SEQUENCING_PRIMER_4_TM': 65.90247007317328}, {})
0 2071
2471
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACGCTG', 'SEQUENCING_PRIMER_1_TM': 61.16590711741475, 'SEQUENCING_PRIMER_2': 'GAG

## 三、基因组测序验证引物
#### 输入：info_input_df, uha_dha_df
#### 输出：genome_sequencing_primer_df

In [16]:
#编辑基因组测序
info_input_df1 = info_input_df[['name','region','seq_uha_max_whole','seq_dha_max_whole','uha_upstream','dha_downstream']].rename(columns={'name':'Name','region':'Region'})
UHA_DHA_df = pd.merge(info_input_df1,uha_dha_df)
UHA_DHA_df['sequencing_template'] =UHA_DHA_df['uha_upstream'] + UHA_DHA_df['seq_uha_max_whole']+UHA_DHA_df['seq_altered']+UHA_DHA_df['seq_dha_max_whole'] + UHA_DHA_df['dha_downstream'] 
UHA_DHA_df['sequencing_region'] = UHA_DHA_df['UHA']+UHA_DHA_df['seq_altered']+UHA_DHA_df['DHA']
UHA_DHA_df['Region'] =  UHA_DHA_df['Name']+';'+UHA_DHA_df['Region']
genome_sequencing_primer_df = p_d_seq.create_sequencing_primer(UHA_DHA_df,sr,'sequencing_template','sequencing_region')

124 2201
2477
({'SEQUENCING_PRIMER_1': 'TCGGATAAGTAGCGAGGAGT', 'SEQUENCING_PRIMER_1_TM': 57.3651846291707, 'SEQUENCING_PRIMER_2': 'CGTTTCCATCTTACACGCGGCC', 'SEQUENCING_PRIMER_2_TM': 64.54478868925031, 'SEQUENCING_PRIMER_3': 'TCGACGTCTGCAGGAAATGGCA', 'SEQUENCING_PRIMER_3_TM': 64.74457393037193, 'SEQUENCING_PRIMER_4': 'ATCGTGCTGATGGCCGACGC', 'SEQUENCING_PRIMER_4_TM': 65.90247007317328}, {})
127 2176
2449
({'SEQUENCING_PRIMER_1': 'GCCTGCTGCTGTTTATCTAGCGT', 'SEQUENCING_PRIMER_1_TM': 63.381286143619604, 'SEQUENCING_PRIMER_2': 'AGTCCGACAATTAGCCTAGGTGA', 'SEQUENCING_PRIMER_2_TM': 60.88051506593223, 'SEQUENCING_PRIMER_3': 'ACGCCGCGAACATCCACACA', 'SEQUENCING_PRIMER_3_TM': 65.40702009302714, 'SEQUENCING_PRIMER_4': 'TTCCCTGATTATGCTGCTGC', 'SEQUENCING_PRIMER_4_TM': 58.31605672573471}, {})
130 2176
2446
({'SEQUENCING_PRIMER_1': 'AATCATCGCACGAGCTGGGG', 'SEQUENCING_PRIMER_1_TM': 62.930384931983724, 'SEQUENCING_PRIMER_2': 'TCACCGGTGCTGGCGTTCTC', 'SEQUENCING_PRIMER_2_TM': 65.14584171422513, 'SEQUENCING

## 四、标准化
##### 输入:uha_primer_df，dha_primer_df，n20up_primer_p_df，plasmid_sequencing_primer_df，genome_sequencing_primer_df，n20down_primer_p_df，seq_altered_primer_df
#### 输出:uha_primer_df,dha_primer_df,sgRNA_primer_df,plasmid_sequencing_primer_df,genome_sequencing_primer_df,plasmid_backbone_p_df,seq_altered_p_df

In [25]:
uha_primer_df,dha_primer_df,n20up_primer_p_df,plasmid_sequencing_primer_df,genome_sequencing_primer_df,n20down_primer_p_df,seq_altered_p_df = su.normal_primer_rename(
                                                                                                                                                        uha_primer_df,
                                                                                                                                                        dha_primer_df,
                                                                                                                                                        n20up_primer_p_df,
                                                                                                                                                        plasmid_sequencing_primer_df,                                                                                                                                                                                                                                                                                                                                                                                                                           
                                                                                                                                                        genome_sequencing_primer_df,
                                                                                                                                                        n20down_primer_p_df,                                                                                                                                         
                                                                                                                                                        seq_altered_primer_df)

Index(['Region', 'primer_f_seq_(5'-3')_joint', 'primer_r_seq_(5'-3')_joint',
       'product_value_joint', 'product_size_joint'],
      dtype='object')


In [24]:
plasmid_backbone_p_df

,Region,PRIMER_LEFT_WHOLE_SEQUENCE,PRIMER_RIGHT_WHOLE_SEQUENCE,PRODUCT_SEQUENCE,PRODUCT_WHOLE_LENGTH
0,153019_ecoil_ybeL_ins;BA000036.3:153019-153020,CCAGGTCTCATTCTGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCATTCTGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711
1,Cgl0006_1176_G_A_sub;BA000036.3:6529-6530,CCAGGTCTCAACCGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCAACCGGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711
2,Cgl0141_cds_del;BA000036.3:153222-155613,CCAGGTCTCAACCGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCAACCGGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711
3,Cgl0591_-1_Ppgk_promoter_ins;BA000036.3:604469...,CCAGGTCTCAGCAGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCAGCAGGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711
4,Cgl0851_ecoli_pgi_sub;BA000036.3:907755-909378,CCAGGTCTCATCCGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCATCCGGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711
5,Cgl1386_327_18to15_sub;BA000036.3:1454441-1454459,CCAGGTCTCAGCTAGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCAGCTAGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711
6,Cgl1436_1113_CAA_del;BA000036.3:1514089-1514092,CCAGGTCTCACGGAGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCACGGAGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711
7,Cgl1790_1647_TCC_sub;BA000036.3:1899017-1899020,CCAGGTCTCAATTAGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCAATTAGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711
8,Cgl2342_213_GCA_ins;BA000036.3:2484874-2484875,CCAGGTCTCATTTCGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCATTTCGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711


## 输出

In [ ]:
#输出引物  
with pd.ExcelWriter(output+'one_plasmid_design_result.xlsx') as writer:  
    uha_primer_df.to_excel(writer,sheet_name = 'Primer_UHA',index_label='No.')
    dha_primer_df.to_excel(writer,sheet_name = 'Primer_DHA',index_label='No.')
    n20up_primer_p_df.to_excel(writer,sheet_name = 'Primer_inserted_fragment',index_label='No.')  
    n20down_primer_p_df.to_excel(writer,sheet_name = 'Primer_plasmid',index_label='No.')
    seq_altered_p_df.to_excel(writer,sheet_name = 'Seq_altered',index_label='No.')
    plasmid_sequencing_primer_df.to_excel(writer,sheet_name = 'Test_primer_P',index_label='No.')
    genome_sequencing_primer_df.to_excel(writer,sheet_name = 'Test_primer_G',index_label='No.')     